In [15]:
import os
import pandas as pd
import re

def extract_clean_content(df, content_col):
    """Lấy nội dung từ cột Nội dung, loại bỏ giá trị rỗng và dấu '-' nếu có."""
    return "\n".join(df[content_col].dropna().astype(str).tolist())

def extract_info_cleaned(df, criteria_col, content_col):
    """Trích xuất thông tin phần 'Thông tin khác' đúng định dạng và đảm bảo không bị lệch dòng."""
    df = df.dropna(subset=[content_col])  # Chỉ kiểm tra nội dung, không cần kiểm tra chỉ tiêu
    df = df.reset_index(drop=True)  # Đảm bảo không có chỉ mục thừa gây lệch
    return "\n".join(df.apply(lambda row: f"{str(row[criteria_col])}: {str(row[content_col])}", axis=1))

def extract_info_cleaned_1(df, criteria_col, content_col):
    """Trích xuất thông tin phần 'Thông tin khác' đúng định dạng và đảm bảo không bị lệch dòng."""
    df[content_col] = df[content_col].shift(-1)
    df = df.dropna(subset=[content_col])
    df = df.reset_index(drop=True)  # Đảm bảo không có chỉ mục thừa gây lệch
    return "\n".join(df.apply(lambda row: f"{str(row[criteria_col])}: {str(row[content_col])}", axis=1))

def convert_xlsx_to_txt(input_folder, output_folder):
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".xlsx"):
            file_path = os.path.join(input_folder, file_name)
            xls = pd.ExcelFile(file_path)
            
            # Đọc các sheet cần thiết
            sheets_data = {sheet: xls.parse(sheet).dropna(how="all").reset_index(drop=True) for sheet in xls.sheet_names}
            
            # Kiểm tra xem sheet 'Thông tin khác' có tồn tại không
            info_other = "Không có thông tin"
            if 'Thông tin khác' in sheets_data:
                extracted_info = extract_info_cleaned_1(sheets_data['Thông tin khác'], sheets_data['Thông tin khác'].columns[1], sheets_data['Thông tin khác'].columns[2])
                if extracted_info.strip():
                    info_other = extracted_info
                    info_other = re.sub(r'nan: |nan: Nội dung|Nội dung','',info_other)
            # Tạo nội dung file TXT
            txt_content = f"""
{str(sheets_data['Sinh cảnh'].iloc[0,0]).replace('- ', '')},{str(sheets_data['Sinh cảnh'].iloc[1,0]).replace('- ', '')},{str(sheets_data['Sinh cảnh'].iloc[2,0]).replace('- ', '')},{str(sheets_data['Sinh cảnh'].iloc[3,0]).replace('- ', '')}

Hình thái:
{extract_clean_content(sheets_data['Hình thái'], sheets_data['Hình thái'].columns[2]).replace('Nội dung','')}

Tập tính:
{extract_clean_content(sheets_data['Tập tính'], sheets_data['Tập tính'].columns[2]).replace('Nội dung','')}

Sinh cảnh:
{extract_clean_content(sheets_data['Sinh cảnh'], sheets_data['Sinh cảnh'].columns[2]).replace('Nội dung','')}

Phân bố:
{extract_clean_content(sheets_data['Phân bố'], sheets_data['Phân bố'].columns[2]).replace('Nội dung','')}

Thông tin khác:
{info_other}
"""
            
            # Xóa dòng trống dư thừa
            txt_content = "\n".join([line for line in txt_content.split("\n") if line.strip()])
            
            # Lưu file TXT
            output_path = os.path.join(output_folder, file_name.replace(".xlsx", ".txt"))
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(txt_content)
            
            print(f"Đã xử lý: {file_name} -> {output_path}")

# Gọi hàm để xử lý tất cả file XLSX trong thư mục
convert_xlsx_to_txt(r"C:\AI_Engeneer\DATN\DATN_data\bird_data_xlsx", r"C:\AI_Engeneer\DATN\DATN_data\fresh_data")


Đã xử lý: 100. Choắt lớn mỏ vàng - Tringa guttifer.xlsx -> C:\AI_Engeneer\DATN\DATN_data\fresh_data\100. Choắt lớn mỏ vàng - Tringa guttifer.txt
Đã xử lý: 101. Choắt mỏ cong bé - Numenius phaeopus.xlsx -> C:\AI_Engeneer\DATN\DATN_data\fresh_data\101. Choắt mỏ cong bé - Numenius phaeopus.txt
Đã xử lý: 102. Choắt mỏ cong hông nâu - Numenius madagascariensis.xlsx -> C:\AI_Engeneer\DATN\DATN_data\fresh_data\102. Choắt mỏ cong hông nâu - Numenius madagascariensis.txt
Đã xử lý: 103. Choắt mỏ cong lớn - Numenius arquata.xlsx -> C:\AI_Engeneer\DATN\DATN_data\fresh_data\103. Choắt mỏ cong lớn - Numenius arquata.txt
Đã xử lý: 104. Choắt mỏ thẳng đuôi vằn - Limosa lapponica.xlsx -> C:\AI_Engeneer\DATN\DATN_data\fresh_data\104. Choắt mỏ thẳng đuôi vằn - Limosa lapponica.txt
Đã xử lý: 105. Choắt mỏ thẳng đuôi đen - Limosa limosa.xlsx -> C:\AI_Engeneer\DATN\DATN_data\fresh_data\105. Choắt mỏ thẳng đuôi đen - Limosa limosa.txt
Đã xử lý: 106. Rẽ khoang - Arenaria interpres.xlsx -> C:\AI_Engeneer\DATN\